# Seq2Seq

이름 : 양주원

기수 : 11기

작성자 : 10 기 신재우

In [1]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 시작 전 실행할 것들

In [2]:
%pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 48.2 MB/s eta 0:00:00


In [3]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pandas as pd
import numpy as np
import tqdm
import spacy
import torchtext
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import random
import os
from ast import literal_eval
import torchtext.vocab as vocab
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

### 경로 설정

In [4]:
path_to_folder = "/content/drive/MyDrive/DSL/[0225]RNN + Transformers_JoowonYang/RNN + Transformers 과제.pdf"

In [5]:
path_train = os.path.join(path_to_folder, "Dataset/train_dataset_Tokenized.csv")
path_val = os.path.join(path_to_folder, "Dataset/val_dataset_Tokenized.csv")
path_test = os.path.join(path_to_folder, "Dataset/test_dataset_Tokenized.csv")
train_dataset = pd.read_csv('/content/drive/MyDrive/DSL/[0225]RNN + Transformers_JoowonYang/Dataset/train_dataset_Tokenized.csv', index_col = 0)
val_dataset = pd.read_csv('/content/drive/MyDrive/DSL/[0225]RNN + Transformers_JoowonYang/Dataset/val_dataset_Tokenized.csv', index_col = 0)
test_dataset = pd.read_csv('/content/drive/MyDrive/DSL/[0225]RNN + Transformers_JoowonYang/Dataset/test_dataset_Tokenized.csv', index_col = 0)

# Dataset 변화 확인

In [6]:
train_dataset.head(3)

원문  \
0  'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...   
1                                       씨티은행에서 일하세요?   
2              푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.   

                                                 번역문  \
0  Bible Coloring' is a coloring application that...   
1                        Do you work at a City bank?   
2  PURITO's bestseller, which recorded 4th rough ...   

                                           en_tokens  \
0  ['<sos>', 'bible', 'coloring', "'", 'is', 'a',...   
1  ['<sos>', 'do', 'you', 'work', 'at', 'a', 'cit...   
2  ['<sos>', 'purito', "'s", 'bestseller', ',', '...   

                                           kr_tokens  
0  ['<sos>', "'", 'Bible', 'Coloring', "'", '은', ...  
1  ['<sos>', '씨티', '은행', '에서', '일하', '세요', '?', '...  
2  ['<sos>', '푸', '리', '토의', '베스트셀러', '는', '해외', ...

# Vocabulary

In [7]:
train_dataset['en_tokens'] = train_dataset['en_tokens'].apply(literal_eval)
train_dataset['kr_tokens'] = train_dataset['kr_tokens'].apply(literal_eval)

val_dataset['en_tokens'] = val_dataset['en_tokens'].apply(literal_eval)
val_dataset['kr_tokens'] = val_dataset['kr_tokens'].apply(literal_eval)

test_dataset['en_tokens'] = test_dataset['en_tokens'].apply(literal_eval)
test_dataset['kr_tokens'] = test_dataset['kr_tokens'].apply(literal_eval)

In [8]:
train_dataset.head(3)

원문  \
0  'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...   
1                                       씨티은행에서 일하세요?   
2              푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.   

                                                 번역문  \
0  Bible Coloring' is a coloring application that...   
1                        Do you work at a City bank?   
2  PURITO's bestseller, which recorded 4th rough ...   

                                           en_tokens  \
0  [<sos>, bible, coloring, ', is, a, coloring, a...   
1  [<sos>, do, you, work, at, a, city, bank, ?, <...   
2  [<sos>, purito, 's, bestseller, ,, which, reco...   

                                           kr_tokens  
0  [<sos>, ', Bible, Coloring, ', 은, 성경, 의, 아름, 답...  
1              [<sos>, 씨티, 은행, 에서, 일하, 세요, ?, <eos>]  
2  [<sos>, 푸, 리, 토의, 베스트셀러, 는, 해외, 에서, 입소문, 만, 으로...

In [9]:
train_dict = train_dataset.to_dict(orient = 'records')
val_dict = val_dataset.to_dict(orient = 'records')
test_dict = test_dataset.to_dict(orient = 'records')

In [10]:
# 하나의 딕셔너리에 한 문장의 세트가 있음 - 원문, 번역문, 원문 토큰, 번역문 토큰
train_dict[0]

{'원문': "'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 앱입니다.",
 '번역문': "Bible Coloring' is a coloring application that allows you to experience beautiful stories in the Bible.",
 'en_tokens': ['<sos>',
  'bible',
  'coloring',
  "'",
  'is',
  'a',
  'coloring',
  'application',
  'that',
  'allows',
  'you',
  'to',
  'experience',
  'beautiful',
  'stories',
  'in',
  'the',
  'bible',
  '.',
  '<eos>'],
 'kr_tokens': ['<sos>',
  "'",
  'Bible',
  'Coloring',
  "'",
  '은',
  '성경',
  '의',
  '아름',
  '답',
  'ㄴ',
  '이야기',
  '를',
  '체험',
  '하',
  'ㄹ',
  '수',
  '있',
  '는',
  '컬러링',
  '앱',
  '이',
  'ㅂ니다',
  '.',
  '<eos>']}

In [11]:
min_freq = 2 # 특정 토큰이 어휘에 포함되기 위해 데이터셋 내에서 등장해야 하는 최소 빈도수
unk_token = "<unk>" # 알 수 없는 단어
pad_token = "<pad>" # 패딩
sos_token = "<sos>" # 문장 시작
eos_token = "<eos>" # 문장 끝

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

In [12]:
# 이터레이터 - data(데이터 셋)의 각 행의 token_field(열 이름) 를 반환
def yield_tokens(data, token_field):
    for item in data:
        yield item[token_field]

# # 예시 : {train_dict}에서 각 행의 {번역문의 토큰들}을 반환
# for i in train_dict:
#   print(i["en_tokens"])

In [13]:
# vocab.build_vocab_from_iterator
#  : 주어진 데이터의 이터레이터를 순회하면서,
#    고유한 토큰들의 집합을 만들고 이를 바탕으로 어휘 객체를 구성하는 함수

# train_dict['en_tokens']의 토큰들을 추출하여 어휘(vocab)를 구축함
en_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "en_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

# train_dict['kr_tokens']의 토큰들을 추출하여 어휘(vocab)를 구축함
kr_vocab = vocab.build_vocab_from_iterator(
    yield_tokens(train_dict, "kr_tokens"),
    specials=special_tokens,
    min_freq=min_freq
)

# set_default_index() :
# Vocab() 객체에 대해 호출됨
# 어휘를 찾을 수 없는 토큰에 대해 반환할 기본 인덱스(이 경우에는, "<unk>")를 설정
en_vocab.set_default_index(en_vocab[unk_token])
kr_vocab.set_default_index(kr_vocab[unk_token])

In [14]:
# Vocab() 내에 포함된 중복되지 않는 단어들의 총
vocab_size_en = len(en_vocab)
vocab_size_kr = len(kr_vocab)
print("English Vocabulary Length : ", vocab_size_en)
print("Korean Vocabulary Length : ", vocab_size_kr)

English Vocabulary Length :  9772
Korean Vocabulary Length :  10442


# 문제 2 (Vocabulary Matching)

토큰화된 단어들을 Vocabulary 과 매칭 시켜서 **벡터**로 바꿔준 뒤에 **텐서**로 바꿔주는 단계까지 포함되어야 합니다!

힌트 :

- $\texttt{en_vocab.lookup_indices}$, 혹은 $\texttt{kr_vocab.lookup_indices}$
- 한줄로 표현이 가능합니다!
- en_indices 와 kr_indices 는 리스트이며, 텐서화 된 것들을 각 토큰마다 리스트로 가져야 합니다!

In [15]:
# 텐서를 저장한 빈 리스트 객체 생성
en_indices = []

# word 내의 단어 토큰의 en_vocab내에서의 인덱스를 찾음(lookup) -> 변환
for word in train_dataset['en_tokens']:

  #lookup_indices() :
  # 리스트(토큰화된 문장, 단어들의 집합)을 input으로 받아,
  # 각 리스트 내의 단어들을 해당 Vocab()(이 경우엔, en_vocab())내의 인덱스로 변환함
  en_indices.append(torch.tensor(en_vocab.lookup_indices(word)))

train_dataset['en_indices'] = en_indices

kr_indices = []
for word in train_dataset['kr_tokens']:

  kr_indices.append(torch.tensor(kr_vocab.lookup_indices(word)))

train_dataset['kr_indices'] = kr_indices

In [16]:
en_indices = []
for word in val_dataset['en_tokens']:

  en_indices.append(torch.tensor(en_vocab.lookup_indices(word)))

val_dataset['en_indices'] = en_indices

kr_indices = []
for word in val_dataset['kr_tokens']:

  kr_indices.append(torch.tensor(kr_vocab.lookup_indices(word)))

val_dataset['kr_indices'] = kr_indices

In [17]:
en_indices = []
for word in test_dataset['en_tokens']:

  en_indices.append(torch.tensor(en_vocab.lookup_indices(word)))

test_dataset['en_indices'] = en_indices

kr_indices = []
for word in test_dataset['kr_tokens']:

  kr_indices.append(torch.tensor(kr_vocab.lookup_indices(word)))

test_dataset['kr_indices'] = kr_indices

# Data Loaders

In [18]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        en_indices = item['en_indices']
        kr_indices = item['kr_indices']
        return en_indices, kr_indices

In [19]:
batch_no = 80

# 배치(batch) 단위로 데이터를 로드할 때 사용
# input 파라메터인 batch에는 batch의 개수가 아니라,
# batch 개수에 해당하는 수의 데이터 샘플 리스트가 들어감
def collate_fn(batch):

    # batch 단위로 (en_indices, kr_indices) 형태의 튜플로 묶음
    en_indices, kr_indices = zip(*batch)

    # batch 내의 인덱스 시퀀스를 패딩하여 모든 시퀀스가 동일한 길이를 갖도록 만듦
    en_indices_padded = pad_sequence(en_indices, batch_first=True, padding_value=0)
    kr_indices_padded = pad_sequence(kr_indices, batch_first=True, padding_value=0)

    return en_indices_padded, kr_indices_padded

train_custom_dataset = CustomDataset(train_dataset[['en_indices', 'kr_indices']])
val_custom_dataset = CustomDataset(val_dataset[['en_indices', 'kr_indices']])
test_custom_dataset = CustomDataset(test_dataset[['en_indices', 'kr_indices']])

# collate_fn : 배치 단위로 데이터를 로드할 때,
# 해당 배치의 데이터를 어떻게 전처리하고 모델에 전달할 준비를 할지를 결정하는 함수
train_loader = DataLoader(train_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_custom_dataset, batch_size=batch_no, shuffle=True, collate_fn=collate_fn)

In [20]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_indices"] for example in batch]
        batch_de_ids = [example["de_indices"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

### 잘 매칭이 되었는지 확인해보기!

In [21]:
for i, (en_indices, kr_indices) in enumerate(train_loader):
    print(f"Batch {i+1}")
    print(f"Shape of en_indices: {en_indices.shape}")
    print(f"Shape of kr_indices: {kr_indices.shape}")

    print("Sample en_indices batch:", en_indices[:1])
    print("Sample kr_indices batch:", kr_indices[:1])

    if i == 1:
        break

Batch 1
Shape of en_indices: torch.Size([80, 27])
Shape of kr_indices: torch.Size([80, 34])
Sample en_indices batch: tensor([[   2,   10,   26,  254,   14, 1834,    9,   12,   10,   26,  123,   40,
         1435,  573,   19,    5,  746,   11,   13,    4,    3,    0,    0,    0,
            0,    0,    0]])
Sample kr_indices batch: tensor([[   2,  816,   13, 2325,   31,    5,   21,   47, 3777, 2032,  615,   36,
            5,  409,    4,    3,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])
Batch 2
Shape of en_indices: torch.Size([80, 36])
Shape of kr_indices: torch.Size([80, 36])
Sample en_indices batch: tensor([[   2,   44,   16,   70,   17,  367,    9, 3661,   20,    3,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])
Sample kr_indices batch: tensor([[   2, 2583, 5421,   27,   65,  160,   

# 문제 3 (Modules without Attention!)

기본적인 Seq2seq 모듈들은 Teacher Forcing 까지 포함해서 진행해주시길 바랍니다.

In [22]:
# 주요 목적 : 입력 시퀀스를 처리하여 고정된 크기의 숨겨진 상태백터 출력하기
class Encoder(nn.Module):
  """
  Encoder 내에서는 Input Language 의 Vocab 크기, Embedding 크기, Hidden 크기가 들어갑니다.

  Embedding 과정에서 Input 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Hidden Size]
  Hidden : [1, Batch_size, Hidden Size]

  힌트 :
  - Batch 가 앞에 있기 때문에 GRU 는 batch_first = True 라는 파라미터를 설정해줘야 합니다.
  - GRU 에서는 Output, Hidden Vector, 총 2개를 뱉어줍니다.
  - GRU 는 nn.GRU, Embedder 는 nn.Embedding 를 사용해주면 됩니다.
  """
  def __init__(self, input_size, embedding_encoder_size, hidden_size):
    super(Encoder, self).__init__()

    self.input_size = input_size
    self.embedding_encoder_size = embedding_encoder_size
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(input_size, embedding_encoder_size) # 임베딩
    self.gru = nn.GRU(embedding_encoder_size, hidden_size, batch_first = True) # Rnn대신 GRU

  def forward(self, input):

    # output: (batch_size, sequence_length, hidden_size(32) x bidirectional(1))
    # hidden: (batch_size, Bidirectional(1) x number of layers(1), hidden_size(32))

    embedded = self.embedding(input) # Step 1 : 임베딩
    output, hidden = self.gru(embedded) # Step 2 : GRU를 통한 시퀀스 처리
    # print("Encoder Hidden shape : " , hidden.shape)
    return output, hidden

In [23]:
class Decoder(nn.Module):
  """
  Decoder 내에서는 Hidden 크기, Embedding Decoder 크기 (Embedding Encoder 크기와 동일), Output Language 의 Vocab 의 크기가 들어갑니다.

  Embedding 과정에서 Output 크기 --> Embedding 크기 로 진행됩니다.
  GRU 내에서는 Embedding 크기 --> Hidden 크기 로 진행됩니다.
  Linear 내에서는 Hidden 크기 --> Output 크기 로 진행됩니다.

  Teacher Forcing 은 디코더를 구성하는 코드에서는 고려를 안해도 되며, 나중에 Seq2Seq 모듈에서 진행해주시길 바랍니다.
  Input 과 Hidden 크기의 경우 Seq2Seq 모듈 내에서 차원을 맞춰주시면 됩니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
    super(Decoder, self).__init__()

    self.embedding_decoder_size = embedding_decoder_size
    self.output_size = output_size
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(output_size, embedding_decoder_size)
    self.gru = nn.GRU(embedding_decoder_size, hidden_size)
    self.linear = nn.Linear(hidden_size, output_size) # Fully connected(Affine) layer

  def forward(self, input, hidden):

    input = input.unsqueeze(0) # input : [batch size] -> [1, batch size] (2차원 텐서)로 차원 증가
    # print("Decoder input.unsqueeze(0) : ", input.shape)

    embedded = self.embedding(input) # [1, batch size, embedding dim]
    # print("after embedding(input) : " , embedded.shape)
    # print("hidden size : ", hidden.shape)

    # output : [1, batch size, hidden dim]
    # hidden : [1, batch size, hidden dim]

    output, hidden = self.gru(embedded, hidden)
    # print('after GRU hidden shape : ', hidden.shape)
    # print('after GRU output shape : ', output.shape)
    preds = self.linear(output.squeeze(0))

    return preds, hidden

In [24]:
class Seq2Seq(nn.Module):
  """
  Seq2Seq 내에서는 Encoder, Decoder, Device 가 들어갑니다.

  Init 단계에서는 정말로 할 것이 크게 없으며, forward 단계에서 for loop 과 if 조건문들을 활용해서 Teacher Forcing 을 적용시켜주면 됩니다.

  Input : [Batch_size, Input Sequences Max Size]
  Target : [Batch_size, Target Sequences Max Size]
  Output : [Batch_size, Input Sequences Max Size, Input Language Vocab Size]

  힌트 :
  - outputs 란 0 으로 채워진 텐서를 하나 만들며 디코더의 t-번째 아웃풀을 outputs 의 t-번째 칸에 넣으면서 for loop 을 진행해주시면 됩니다.
  - Target 의 첫번째 요소 (:, 0) 는 항상 <sos> 입니다! 이것을 디코더의 첫 Input 으로 넣어준 다음에 for loop 을 진행해주시면 됩니다.
  - random.random() < teacher_forcing_ratio 를 통해서 조건문을 만들 수가 있습니다!
  - Teacher Force 가 적용이 안될 때는디코더의 아웃풋에는 모든 토큰들에 대한 점수가 나오게 됩니다.
    이것들 중에서 가장 최댓값만 중요하기에 argmax 를 이용해서 최댓값의 인덱스를 다음 Decoder 의 Input 으로 넣어주면 됩니다.
  - squeeze 와 unsqueeze 를 적극적으로 활용해서 차원을 조정해주시면 됩니다!
  """
  def __init__(self, encoder, decoder, device):
    super(Seq2Seq, self).__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, input, target, teacher_forcing_ratio = 0.5):
    # input = [단어의 개수, 배치 사이즈]
    # target = [단어의 개수, 배치 사이즈]

    # Decoder의 최종 결과를 담을 텐서 객체 만들기
    batch_size = target.shape[0] # 배치 크기
    target_len = target.shape[1] # 단어 개수
    target_vocab_size = self.decoder.output_size

    # Decoder 결과를 저장할 텐서
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)
    output, hidden = self.encoder(input)

    # Decoder에 들어갈 첫 input의 <sos> 토큰
    input = target[: ,0]

    for t in range(1, target_len):
      output, hidden = self.decoder(input, hidden)
      outputs[t] = output

      # 가장 높은 확률로 예측된 토큰
      top1 = output.argmax(1)

      # 0.5의 확률
      teacher_force = random.random() < teacher_forcing_ratio

      input = target[ :, t] if teacher_force else top1

    return outputs

# Training

In [25]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

### 모델 확인 + Initialize 시키기

In [26]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 30,502,090 trainable parameters


### 하이퍼 파라미터 설정!

In [27]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

## Training 코드!

In [28]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss

    model.eval()
    epoch_loss = 0
    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()

    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 3.715059756596883, Val Loss: 3.6478755511934797
Epoch: 2, Train Loss : 3.6600274098714194, Val Loss: 3.6011369379739913
Epoch: 3, Train Loss : 3.6358217531840005, Val Loss: 3.5996956598191034
Epoch: 4, Train Loss : 3.655147493362427, Val Loss: 3.6099721704210554
Epoch: 5, Train Loss : 3.648541255950928, Val Loss: 3.582700426616366
Epoch: 6, Train Loss : 3.6507488600413005, Val Loss: 3.628928498616294
Epoch: 7, Train Loss : 3.6446830457051593, Val Loss: 3.5829362528664723
Epoch: 8, Train Loss : 3.627137968699137, Val Loss: 3.6166780562627885
Epoch: 9, Train Loss : 3.6412589906056723, Val Loss: 3.5973216011410667
Epoch: 10, Train Loss : 3.6251562671661377, Val Loss: 3.574034615168496


파라미터 저장

In [29]:
path_to_folder2 = "/content/drive/MyDrive/DSL/[0225]RNN + Transformers_JoowonYang"
path_to_save = os.path.join(path_to_folder2, "model_without_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [30]:
test_batch_en, test_batch_kr = next(iter(test_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', 'then', 'can', 'i', 'keep', 'working', 'on', 'it', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '그러', '면', '저', '가', '그', '일', '을', '계속', '하', 'ㄹ', '수', '있', '나요', '?', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<un

\<unk\> 는 저희 Vocabulary 에는 존재하지 않는 토큰들을 의미합니다.

아무리 epoch 를 늘리거나 무엇을 해보려고 해도 완전히 좋은 결과가 나오지는 않을 것입니다. 해당 이유는 Attention 이 적용이 안되었기 때문이며, 다음과 같이 Attention 을 적용시켜서 한번 진행하면 결과가 다르다는 것을 알 수가 있습니다.

# 문제 4 (Modules WITH Attention)

만약에 메모리가 다 찼다면 해당 Run 을 Reconnect 시켜서 위의 Output 만 남긴 채로 제출하시면 됩니다.

**이렇게 진행하게 된다면 당연히 문제 3 이전의 코드들은 다시 돌려야 됩니다!!**

In [31]:
class Encoder(nn.Module):
  """
  이전과 Encoder 부분은 달라지는게 없습니다!
  """
  def __init__(self, input_size, embedding_encoder_size, hidden_size):

    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, embedding_encoder_size)
    self.gru = nn.GRU(embedding_encoder_size, hidden_size, batch_first = True)

  def forward(self, input):

    # print("(1) Encoder_forward_input : ", input)

    embedded = self.embedding(input)
    # print("(2) Encoder_forward_embedded : ", embedded)

    output, hidden = self.gru(embedded)

    return output, hidden

In [32]:
class Attention(nn.Module):
  """
  해당 모듈에서는 Attention Value 까지 찾는 단계입니다.

  Encoder 내에서의 Output 들의 개수마다 for loop 을 진행시켜서 합산시켜서 Attention Value 를 구하는 방법도 있지만,
  Encoder Output 들을 모두 Matrix 으로 표현해서 구하는 방법도 있습니다.

  Decoder_Outputs : [Batch_size, 1, Hidden_size]
  Encoder_Outputs : [Batch_size, Sequence Length, Hidden_size]

  힌트 :
  - Decoder_Outputs 의 차원이 위와 같이 안 나온다면 차원 조정을 위해서 squeeze, unsqueeze, transpose 등의 함수들을 활용해주면 됩니다.
  - Matrix Multiplication 을 진행할 때에 Batch 까지 곱해지는 것을 방지하기 위해서 torch.bmm(x, y) 를 활용해주면 됩니다.
  - 곱해줄 때에는 그저 x * y 를 해주면 됩니다.
  - 합산할 때에는 배치도 고려해야 함으로 torch.sum(x, dim = 1) 을 활용해주면 됩니다.
  - 해당 모듈 내에서 필요한 nn. 함수는 오로지 Softmax 밖에 없습니다.
  """

  def __init__(self, hidden_size):

    super(Attention, self).__init__()
    self.hidden_size = hidden_size

  def forward(self, decoder_outputs, encoder_outputs):

    # [1, batch_size, hidden_size] -> [batch_size, 1, hidden_size]
    decoder_outputs = decoder_outputs.transpose(0,1)
    # print("(1) Attention_decoder_outputs.shape : " ,  decoder_outputs.shape)
    # print("(2) Attention_encoder_outputs.transpose(1,2).shape : " , encoder_outputs.transpose(1,2).shape)

    # [batch_size, 1, seq_len]
    attention_scores = torch.bmm(decoder_outputs, encoder_outputs.transpose(1, 2))


    # 소프트맥스 함수를 사용 -> 어텐션 스코어 정규화
    # [batch_size, 1, seq_len]
    attention_weights = F.softmax(attention_scores, dim=-1)

    # 어텐션 가중치와 인코더의 아웃풋을 곱하여 컨텍스트 벡터를 계산합니다.
    # [batch_size, 1, hidden_size]
    # print("(1) Attention_attention_weights_type : ", type(attention_weights))
    # print("(2) Attention_encoder_outputs_type : ", type(encoder_outputs))
    context_vectors = torch.bmm(attention_weights, encoder_outputs)

    return context_vectors, attention_weights

In [33]:
class Decoder(nn.Module):
  """
  Attention Value 를 Attention 모듈을 통해서 구했기 때문에 Concatenate, Tangent Hyperbolic 등의 것들이 여기에서 적용이 됩니다.

  힌트 :
  - 모든 것을 진행하기 전에 앞서서 먼저 GRU 를 지나고 나서 Output 을 활용해야 합니다.
  - Attention 모듈도 사용해야 하기 때문에 해당 모듈을 현 모듈에서 정의도 해줘야 합니다.
  - Concatenate 의 경우 torch.cat((x, y), dim = 1) 형태로 사용하면 됩니다.
  - W_c 와 v_t 를 곱해줄 때 W_c 는 가중치 행렬이며, 이것은 그저 nn.Linear 형태로 표현이 가능합니다.
  """
  def __init__(self, hidden_size, embedding_decoder_size, output_size):
    super(Decoder, self).__init__()

    self.hidden_size = hidden_size
    self.embedding_decoder_size = embedding_decoder_size
    self.output_size = output_size

    self.embedding = nn.Embedding(output_size, embedding_decoder_size)
    self.gru = nn.GRU(embedding_decoder_size + hidden_size, hidden_size, batch_first=True)
    self.fc = nn.Linear(hidden_size * 2, output_size)
    self.attention = Attention(hidden_size)
    self.softmax = nn.Softmax(dim = 1)

  def forward(self, input, hidden, encoder_outputs):

    # input : [batch size] -> [batch size, 1]
    input = input.unsqueeze(1)

    # 임베딩 레이어 통과
    # embedded : [batch size, 1, embedding_dim]
    embedded = self.embedding(input)

    # 어텐션 계산
    context_vector, attention_weights = self.attention(hidden, encoder_outputs)

    # print("(1) Decoder_attention_weights_type : ", type(attention_weights))
    # print("(2) Decoder_encoder_outputs_type : ", type(encoder_outputs))

    # # 어텐션 가중치와 인코더 출력을 이용해 컨텍스트 벡터를 생성
    # context_vector = torch.bmm(attention_weights, encoder_outputs)

    # print("(1) Decoder_embedded_type&shape: ", type(embedded), embedded.shape)
    # print("(2) Decoder_encoder_outputs_type&shape : ", type(encoder_outputs), encoder_outputs.shape)

    # GRU로 전달하기 전에 임베딩과 컨텍스트 벡터를 연결
    gru_input = torch.cat((embedded, context_vector), dim=2)

    # GRU로 전달하기 전에 hidden의 구성 위치를 바꿈
    # print("(1) Decoder_concat_b4_gru_input : ", gru_input.shape)
    # print("(2) Decoder_hidden[-1]_b4_gru_input : ", hidden)

    # GRU 통과
    output, hidden = self.gru(gru_input, hidden)

    # print("(3) Decoder_after_gru_output : ", output.shape)
    # print("(4) Decoder_after_gru_hidden : ", hidden.shape)

    # 출력을 선형 레이어에 통과시켜 예측 생성
    output = output.squeeze(0) # [batch size, hidden_dim]
    # print("(5) Decoder_after_output.squeeze(0)" , output.shape) # torch.Size([80, 1, 1024])

    context_vector = context_vector.squeeze(0)
    # print("(6) Decoder_after_context_vector" , context_vector.shape) # torch.Size([80, 1, 1024])

    output = torch.cat((output, context_vector), dim=2)

    # print("(7) Decoder_b4_linear_fclayer : ", output.shape)
    output = self.fc(output)
    output = self.softmax(output)

    output = output.squeeze()

    return output, hidden, attention_weights

In [34]:
class Seq2Seq(nn.Module):
  """
  이전과 Seq2Seq 부분은 크게 달라지는게 없습니다!
  단지 디코더가 Input 을 하나 더 받기 때문에 조심해주시길 바랍니다.
  """
  def __init__(self, encoder, decoder):
    super(Seq2Seq, self).__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, input, target, teacher_forcing_ratio = 0.5):

    batch_size = target.shape[0]
    target_len = target.shape[1]
    target_vocab_size = self.decoder.output_size

    # 각 시퀀스의 예측을 저장할 텐서를 초기화합니다
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)


    # 인코더를 통해 입력 시퀀스를 처리하고 문맥 벡터를 얻습니다
    encoder_output, hidden = self.encoder(input)

    # 디코더의 첫 입력은 일반적으로 <sos> 토큰입니다
    input = target[:, 0]

    for t in range(1, target_len):
        # 디코더에 현재 입력과 숨겨진 상태를 전달합니다
        # 인코더 출력도 어텐션 메커니즘에 사용하기 위해 전달합니다
        output, hidden, _ = self.decoder(input, hidden, encoder_output)

        # 예측을 저장합니다
        outputs[t] = output

        # teacher forcing을 사용할 지 여부를 결정합니다
        teacher_force = random.random() < teacher_forcing_ratio

        # 다음 입력을 얻습니다: teacher forcing을 사용하면 실제 목표 출력을 사용하고,
        # 그렇지 않으면 이전 예측을 사용합니다
        input = (target[:, t] if teacher_force else output.argmax(1))

    return outputs

# Training

In [35]:
input_dim = len(en_vocab)
output_dim = len(kr_vocab)
encoder_embedding_dim = 512
decoder_embedding_dim = 512
hidden_dim = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim)
decoder = Decoder(hidden_dim, decoder_embedding_dim, output_dim)
model = Seq2Seq(encoder, decoder).to(device)

### 모델 확인 + Initialize 시키기

In [36]:
model.apply(init_weights)
print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 44,340,426 trainable parameters


### 하이퍼 파라미터 설정!

In [37]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

n_epochs = 10
teacher_forcing_ratio = 0.5

### 학습 시키기!

In [38]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    for (en_indices, kr_indices) in train_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        optimizer.zero_grad()

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio)  # Target 에서는 <eos> 는 무시하고 진행합니다.
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)

        kr_indices = kr_indices[:, 1:].contiguous().view(-1)  # Target 에서 <sos> 를 무시하기 위함입니다.

        loss = criterion(output, kr_indices)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    train_loss = epoch_loss
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
      for (en_indices, kr_indices) in val_loader:
        en_indices, kr_indices = en_indices.to(device), kr_indices.to(device)

        output = model(en_indices, kr_indices[:, :-1], teacher_forcing_ratio = 0.0)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        kr_indices = kr_indices[:, 1:].contiguous().view(-1)

        loss = criterion(output, kr_indices)
        epoch_loss += loss.item()
    print(f'Epoch: {epoch+1}, Train Loss : {train_loss / len(train_loader)}, Val Loss: {epoch_loss / len(val_loader)}')

Epoch: 1, Train Loss : 9.253597089131674, Val Loss: 9.253597062731545
Epoch: 2, Train Loss : 9.253597053527832, Val Loss: 9.253597108144609
Epoch: 3, Train Loss : 9.253597040812174, Val Loss: 9.25359697190542
Epoch: 4, Train Loss : 9.253596989949544, Val Loss: 9.25359666915167
Epoch: 5, Train Loss : 9.253597048441568, Val Loss: 9.253596684289358
Epoch: 6, Train Loss : 9.253596989949544, Val Loss: 9.253596941630045
Epoch: 7, Train Loss : 9.253597183227539, Val Loss: 9.253596956767733
Epoch: 8, Train Loss : 9.253597071329752, Val Loss: 9.253596865941608
Epoch: 9, Train Loss : 9.253597104390462, Val Loss: 9.253596654013982
Epoch: 10, Train Loss : 9.253597030639648, Val Loss: 9.253597198970734


### 파라미터 저장

In [40]:
path_to_folder3 = '/content/drive/MyDrive/DSL/[0225]RNN + Transformers_JoowonYang'
path_to_save = os.path.join(path_to_folder3, "model_WITH_attention.pt")
torch.save(model.state_dict(), path_to_save)

### 적용시켜보기

In [41]:
test_batch_en, test_batch_kr = next(iter(train_loader))
test_en = test_batch_en[0].numpy()
test_kr = test_batch_kr[0].numpy()
model.eval()
with torch.no_grad():
  output = model(test_batch_en.to(device), test_batch_kr.to(device)[:, :-1], teacher_forcing_ratio = 0.0)
  print(en_vocab.lookup_tokens(test_en))
  print(kr_vocab.lookup_tokens(test_kr))
  print(kr_vocab.lookup_tokens(output.argmax(2).cpu().numpy()[0]))

['<sos>', '<unk>', 'president', 'has', 'the', 'korea', 'exchange', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<sos>', '<unk>', '사장', '이', '한국', '거래소', '가지', '고', '있', '어요', '.', '<eos>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<

아마 결과가 온전히 잘 나오지는 않을 겁입니다. 해당 이유는 단순히 파라미터 개수가 적기 때문이며, 잘 나오기 위해서는 이보다 훨씬 늘려야 합니다. 원하신다면 Hidden_size, Embedding_size 를 늘려서 진행하는 것도 가능하긴 합니다!

과제 하시느라 정말로 고생하셨습니다!